# **AI for Finance - Assignment 2**

**Francesco Trono (221723)**

*University of Trento - A.Y. 2021/2022*

francesco.trono@studenti.unitn.it

Click here to open & run in Google Colab environment:


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ftrono/AI_for_Finance/blob/main/Assignment_2/221723_Assignment_2.ipynb)

°  °  °

In this Assignment, I have performed an OLS Regression of a stock and a portfolio, using different source data, in order to calculate their performance compared to the market.

This *Jupyter* notebook contains the code split into separate cells, each visualizing also the output of each execution, the exhibits, the plots and the comments. The full guide to the notebook is available in the submitted PDF report.

**PART I** of the notebook contains all the data extraction and preparation operations, while **PART II** directly uses the prepared data to answer the Assignment questions.

#### **Libraries import:**

In [1]:
!pip3 install --upgrade pandas-datareader >/dev/null
!pip3 install fredpy
#imports:
import os, math, statistics, fredpy
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
fredpy.api_key = 'c8c6b2971c87b7187b1c5989c1a89008'

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


---

## **Question 1**

#### **Get Fama & French data:**
Risk-free (Rf) rates + 3 factors data (Market, SMB, HML).

In [2]:
#download risk free file from Fama&French website, extract csv and import it to Pandas:
zipname = 'F-F_Research_Data_Factors_CSV.zip'
csvname = r"F-F_Research_Data_Factors.CSV"
if os.path.exists(zipname) == False:
  !wget 'http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip' >/dev/null
  !unzip 'F-F_Research_Data_Factors_CSV.zip' >/dev/null

  #extract needed exhibit from csv file (there are two exhibits):
  with open(csvname, 'r+') as fp:
    #store read file:
    lines = fp.readlines()  

    #skip first three lines, then save index of the first empty line (at the end of the first exhibit):
    ind = 3
    for line in lines[3:-1]:
      ind += 1
      if line == '\n':
        print(ind)
        break

    #empty original file:
    fp.seek(0)
    fp.truncate()
    #rewrite only lines corresponding to the 1st exhibit:
    fp.writelines(lines[3:ind])

#import csv to Pandas:
FF_30 = pd.read_csv(csvname)
FF_30.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
#slice needed portion of data (max = 30Y):
FF_30 = FF_30.loc[(FF_30['Date'] >= 199202)]
FF_30 = FF_30.loc[(FF_30['Date'] <= 202201)]
FF_30.reset_index(drop=True, inplace=True)
print(FF_30)

--2022-03-29 13:09:59--  http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip
Resolving mba.tuck.dartmouth.edu (mba.tuck.dartmouth.edu)... 129.170.171.144
Connecting to mba.tuck.dartmouth.edu (mba.tuck.dartmouth.edu)|129.170.171.144|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12579 (12K) [application/x-zip-compressed]
Saving to: ‘F-F_Research_Data_Factors_CSV.zip’

F-F_Research_Data_F 100%[===================>]  12.28K  --.-KB/s    in 0s      

2022-03-29 13:09:59 (235 MB/s) - ‘F-F_Research_Data_Factors_CSV.zip’ saved [12579/12579]

1153
       Date  Mkt-RF   SMB    HML    RF
0    199202    1.09  0.87   6.47  0.28
1    199203   -2.66 -1.04   3.55  0.34
2    199204    1.07 -6.06   4.32  0.32
3    199205    0.30  0.41   1.19  0.28
4    199206   -2.34 -3.07   3.25  0.32
..      ...     ...   ...    ...   ...
355  202109   -4.37  0.80   5.09  0.00
356  202110    6.65 -2.28  -0.44  0.00
357  202111   -1.55 -1.35  -0.53  0.

#### **Get Microsoft data (Rs):**

In [3]:
#download MSFT data (max = 30Y):
Msft_30 = pdr.get_data_yahoo('MSFT', start='1992-02-01', end='2022-01-31', interval='m')
Msft_30.reset_index(inplace=True)
print(Msft_30[['Date', 'Close', 'Adj Close']])

          Date       Close   Adj Close
0   1992-02-01    2.572917    1.625809
1   1992-03-01    2.468750    1.559986
2   1992-04-01    2.296875    1.451380
3   1992-05-01    2.520833    1.592897
4   1992-06-01    2.187500    1.382267
..         ...         ...         ...
355 2021-09-01  281.920013  280.824524
356 2021-10-01  331.619995  330.331360
357 2021-11-01  330.589996  329.305359
358 2021-12-01  336.320007  335.626038
359 2022-01-01  310.980011  310.338318

[360 rows x 3 columns]


In [4]:
#convert MSFT prices to returns (use monthly pct_change):
ret_msft = Msft_30['Adj Close'].pct_change(1, fill_method ='ffill')
#replace NaN with zeroes:
ret_msft.fillna(0, inplace=True)
print(ret_msft.head())

0    0.000000
1   -0.040486
2   -0.069620
3    0.097506
4   -0.132231
Name: Adj Close, dtype: float64


#### **Get portfolio data (Rp):**

Using the same portfolio already created for Assignment 1 ([click here for full Portfolio description](https://github.com/ftrono/AI_for_Finance/blob/main/Assignment_1/221723_Assignment_1.ipynb)).

Here follows a recap table:



|	STOCK	|	TICKER	|	INDUSTRY	|	MARKET	|	COUNTRY	|
|	---------	|	---------	|	---------	|	---------	|	---------	|
|	*Microsoft*	|	***MSFT***	|	*tech*	|	*Nasdaq*	|	*USA*	|
|	Boeing	|	**BA**	|	aerospace	|	NYSE	|	USA	|
|	Southwest Airlines	|	**LUV**	|	airways	|	NYSE	|	USA	|
|	Ford Motors	|	**F**	|	automotive	|	NYSE	|	USA	|
|	Barclays	|	**BCS**	|	banking	|	NYSE	|	UK	|
|	Coca-Cola	|	**KO**	|	beverages	|	NYSE	|	USA	|
|	Walmart	|	**WMT**	|	consumer stores	|	NYSE	|	USA	|
|	Unilever	|	**UL**	|	consumer, household	|	NYSE	|	UK	|
|	Johnson & Johnson	|	**JNJ**	|	healthcare, pharma	|	NYSE	|	USA	|
|	Pfizer	|	**PFE**	|	healthcare, pharma	|	NYSE	|	USA	|
|	The Home Depot	|	**HD**	|	home improvement, stores	|	NYSE	|	USA	|
|	3M	|	**MMM**	|	industrial	|	NYSE	|	USA	|
|	Aegon N.V.	|	**AEG**	|	insurance	|	NYSE	|	Netherlands	|
|	JPMorgan Chase	|	**JPM**	|	investment banking	|	NYSE	|	USA	|
|	TotalEnergies SE	|	**TTE**	|	o&g	|	NYSE	|	France	|
|	Canadian Pacific Railway	|	**CP**	|	railroads	|	NYSE	|	Canada	|
|	Canon	|	**CAJ**	|	tech	|	NYSE	|	Japan	|
|	Sony	|	**SONY**	|	tech	|	NYSE	|	Japan	|
|	Ericsson	|	**ERIC**	|	tech communications	|	Nasdaq	|	Sweden	|
|	Vodafone	|	**VOD**	|	tlc	|	Nasdaq	|	UK	|
|	Telefónica	|	**TEF**	|	tlc	|	NYSE	|	Spain	|



In [5]:
tickers = ['MSFT', 'BA', 'LUV', 'F', 'BCS', 'KO', 'WMT', 'UL', 'JNJ', 'PFE', 'HD', 'MMM', 'AEG', 'JPM', 'TTE', 'CP', 'CAJ', 'SONY', 'ERIC', 'VOD', 'TEF']

#download Portfolio data (max = 30Y):
MyPortfolio = pdr.get_data_yahoo(tickers, start='1992-02-01', end='2022-01-31', interval='m')
MyPortfolio.reset_index(inplace=True)
MyPortfolio = MyPortfolio[['Date', 'Adj Close']]
print(MyPortfolio.head())

Attributes       Date Adj Close                                           \
Symbols                    MSFT         BA       LUV         F       BCS   
0          1992-02-01  1.625809  12.904117  2.343874  2.040116  2.054783   
1          1992-03-01  1.559986  12.370317  2.240678  2.130326  1.725619   
2          1992-04-01  1.451380  12.405565  2.352340  2.518919  2.054304   
3          1992-05-01  1.592898  12.370317  2.568220  2.527457  2.282560   
4          1992-06-01  1.382267  11.306597  2.553331  2.612891  2.152500   

Attributes                                          ...                       \
Symbols           KO       WMT        UL       JNJ  ...        MMM       AEG   
0           5.016705  8.568611  4.016265  6.248079  ...   9.835167  1.041640   
1           5.016705  8.449602  3.765247  6.039351  ...   9.676134  1.049188   
2           5.111558  8.299171  3.955411  5.876751  ...  10.221266  1.045414   
3           5.419480  8.418297  4.094888  6.000637  ...  10.425693 

In [6]:
#convert portfolio assets' prices to returns (use monthly pct_change):
ret_mp = MyPortfolio['Adj Close'].pct_change(1, fill_method ='ffill')
#replace NaN with zeroes:
ret_mp.fillna(0, inplace=True)
print(ret_mp.head())

Symbols      MSFT        BA       LUV         F       BCS        KO       WMT  \
0        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1       -0.040486 -0.041367 -0.044028  0.044218 -0.160194  0.000000 -0.013889   
2       -0.069620  0.002849  0.049834  0.182410  0.190474  0.018908 -0.017803   
3        0.097506 -0.002841  0.091773  0.003389  0.111111  0.060240  0.014354   
4       -0.132231 -0.085990 -0.005797  0.033802 -0.056980 -0.090909  0.014151   

Symbols        UL       JNJ       PFE  ...       MMM       AEG       JPM  \
0        0.000000  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
1       -0.062500 -0.033407 -0.062109  ... -0.016170  0.007246 -0.026022   
2        0.050505 -0.026923  0.028777  ...  0.056338 -0.003597  0.082257   
3        0.035262  0.021081  0.043707  ...  0.020000 -0.025271  0.039146   
4        0.069418 -0.071633 -0.016454  ...  0.023017  0.058800  0.017123   

Symbols       TTE        CP       CAJ      SONY      ERI

#### **Get additional factors data:**
A) TERM: Maturity spread (10Y - 3M Treasury bonds):

In [7]:
#Government spread 10Y vs 3Y is already available as series in FRED database:
Term = fredpy.series("T10Y3M")
Term = Term.as_frequency(freq="M", method='last')
Term = Term.window(['1992-02-01', '2022-01-31'])
Term = pd.DataFrame(data={'Term': Term.data})
Term.reset_index(inplace=True)
Term

,date,Term
0,1992-02-01,3.24
1,1992-03-01,3.39
2,1992-04-01,3.82
3,1992-05-01,3.54
4,1992-06-01,3.49
...,...,...
355,2021-09-01,1.48
356,2021-10-01,1.50
357,2021-11-01,1.38
358,2021-12-01,1.46


B) CREDIT: Corporate credit spread (BAA - AAA corporate bond yields):

In [8]:
#Get AAA and BAA series from FRED database, then calculate the spread:
AAA = fredpy.series("AAA")
AAA = AAA.window(['1992-02-01', '2022-01-31'])
BAA = fredpy.series("BAA")
BAA = BAA.window(['1992-02-01', '2022-01-31'])
Credit = pd.DataFrame({'r_AAA': AAA.data, 'r_BAA': BAA.data, '(BAA-AAA)': BAA.data - AAA.data})
Credit.reset_index(inplace=True)
Credit

,date,r_AAA,r_BAA,(BAA-AAA)
0,1992-02-01,8.29,9.23,0.94
1,1992-03-01,8.35,9.25,0.90
2,1992-04-01,8.33,9.21,0.88
3,1992-05-01,8.28,9.13,0.85
4,1992-06-01,8.22,9.05,0.83
...,...,...,...,...
355,2021-09-01,2.53,3.23,0.70
356,2021-10-01,2.68,3.35,0.67
357,2021-11-01,2.62,3.28,0.66
358,2021-12-01,2.65,3.30,0.65


### **Merge data together:**

In [9]:
'''
Merged DataFrame with all returns (30Y):
- Fama & French factors data (Rf, (Rm-Rf), SMB, HML)
- Additional factors data: Term, Credit
- 'Rs_adj': monthly returns of MFST stock
- 'Rp_adj: monthly returns of Portfolio (columnwise average of returns of all Portfolio assets, equally weighted as per Task requirement)
- Excess returns
'''
Data_30 = pd.DataFrame(data={'Date': FF_30['Date'], 'Rf': FF_30['RF']/100, 'Mkt_prem': FF_30['Mkt-RF']/100, 'SMB': FF_30['SMB']/100, 'HML': FF_30['HML']/100, 'Term': Term['Term']/100, 'Credit': Credit['(BAA-AAA)']/100, 'Rs_adj': ret_msft, 'Rp_adj': ret_mp.mean(axis = 1)})
Data_30['(Rs-Rf)'] = Data_30['Rs_adj'] - Data_30['Rf']
Data_30['(Rp-Rf)'] = Data_30['Rp_adj'] - Data_30['Rf']
Data_30

,Date,Rf,Mkt_prem,SMB,HML,Term,Credit,Rs_adj,Rp_adj,(Rs-Rf),(Rp-Rf)
0,199202,0.0028,0.0109,0.0087,0.0647,0.0324,0.0094,0.000000,0.000000,-0.002800,-0.002800
1,199203,0.0034,-0.0266,-0.0104,0.0355,0.0339,0.0090,-0.040486,-0.032811,-0.043886,-0.036211
2,199204,0.0032,0.0107,-0.0606,0.0432,0.0382,0.0088,-0.069620,0.043117,-0.072820,0.039917
3,199205,0.0028,0.0030,0.0041,0.0119,0.0354,0.0085,0.097506,0.054829,0.094706,0.052029
4,199206,0.0032,-0.0234,-0.0307,0.0325,0.0349,0.0083,-0.132231,-0.027154,-0.135431,-0.030354
...,...,...,...,...,...,...,...,...,...,...,...
355,202109,0.0000,-0.0437,0.0080,0.0509,0.0148,0.0070,-0.064331,-0.013017,-0.064331,-0.013017
356,202110,0.0000,0.0665,-0.0228,-0.0044,0.0150,0.0067,0.176291,0.036900,0.176291,0.036900
357,202111,0.0000,-0.0155,-0.0135,-0.0053,0.0138,0.0066,-0.003106,-0.021692,-0.003106,-0.021692
358,202112,0.0001,0.0310,-0.0157,0.0323,0.0146,0.0065,0.019194,0.053288,0.019094,0.053188


In [10]:
#Split into smaller slices (20Y, 10Y, 5Y):
Data_20 = Data_30.loc[(Data_30['Date'] >= 200202)]
Data_20.reset_index(drop=True, inplace=True)
Data_10 = Data_30.loc[(Data_30['Date'] >= 201202)]
Data_10.reset_index(drop=True, inplace=True)
Data_5 = Data_30.loc[(Data_30['Date'] >= 201702)]
Data_5.reset_index(drop=True, inplace=True)
#print:
Data_20

,Date,Rf,Mkt_prem,SMB,HML,Term,Credit,Rs_adj,Rp_adj,(Rs-Rf),(Rp-Rf)
0,200202,0.0013,-0.0229,-0.0100,0.0216,0.0309,0.0138,-0.084288,0.004408,-0.085588,0.003108
1,200203,0.0013,0.0424,0.0421,0.0107,0.0363,0.0130,0.033768,0.036194,0.032468,0.034894
2,200204,0.0015,-0.0520,0.0596,0.0388,0.0334,0.0127,-0.133477,-0.029138,-0.134977,-0.030638
3,200205,0.0014,-0.0138,-0.0321,0.0153,0.0334,0.0134,-0.025832,-0.014578,-0.027232,-0.015978
4,200206,0.0013,-0.0721,0.0428,-0.0003,0.0316,0.0132,0.074445,-0.048670,0.073145,-0.049970
...,...,...,...,...,...,...,...,...,...,...,...
235,202109,0.0000,-0.0437,0.0080,0.0509,0.0148,0.0070,-0.064331,-0.013017,-0.064331,-0.013017
236,202110,0.0000,0.0665,-0.0228,-0.0044,0.0150,0.0067,0.176291,0.036900,0.176291,0.036900
237,202111,0.0000,-0.0155,-0.0135,-0.0053,0.0138,0.0066,-0.003106,-0.021692,-0.003106,-0.021692
238,202112,0.0001,0.0310,-0.0157,0.0323,0.0146,0.0065,0.019194,0.053288,0.019094,0.053188


In [11]:
Data_10

,Date,Rf,Mkt_prem,SMB,HML,Term,Credit,Rs_adj,Rp_adj,(Rs-Rf),(Rp-Rf)
0,201202,0.0000,0.0442,-0.0186,0.0043,0.0190,0.0129,0.074839,0.036516,0.074839,0.036516
1,201203,0.0000,0.0311,-0.0066,0.0112,0.0216,0.0124,0.023074,0.020922,0.023074,0.020922
2,201204,0.0000,-0.0085,-0.0041,-0.0077,0.0185,0.0123,-0.007439,-0.032458,-0.007439,-0.032458
3,201205,0.0001,-0.0619,0.0009,-0.0106,0.0152,0.0127,-0.088382,-0.082907,-0.088482,-0.083007
4,201206,0.0000,0.0389,0.0064,0.0059,0.0158,0.0138,0.054838,0.058235,0.054838,0.058235
...,...,...,...,...,...,...,...,...,...,...,...
115,202109,0.0000,-0.0437,0.0080,0.0509,0.0148,0.0070,-0.064331,-0.013017,-0.064331,-0.013017
116,202110,0.0000,0.0665,-0.0228,-0.0044,0.0150,0.0067,0.176291,0.036900,0.176291,0.036900
117,202111,0.0000,-0.0155,-0.0135,-0.0053,0.0138,0.0066,-0.003106,-0.021692,-0.003106,-0.021692
118,202112,0.0001,0.0310,-0.0157,0.0323,0.0146,0.0065,0.019194,0.053288,0.019094,0.053188


In [12]:
Data_5.head()

,Date,Rf,Mkt_prem,SMB,HML,Term,Credit,Rs_adj,Rp_adj,(Rs-Rf),(Rp-Rf)
0,201702,0.0004,0.0357,-0.0204,-0.0167,0.0183,0.0069,-0.010364,0.043817,-0.010764,0.043417
1,201703,0.0003,0.0017,0.0113,-0.0333,0.0164,0.0067,0.035625,0.015175,0.035325,0.014875
2,201704,0.0005,0.0109,0.0072,-0.0213,0.0149,0.0070,0.039478,0.019800,0.038978,0.019300
3,201705,0.0006,0.0106,-0.0252,-0.0375,0.0123,0.0070,0.020158,0.030997,0.019558,0.030397
4,201706,0.0006,0.0078,0.0223,0.0149,0.0128,0.0069,-0.007373,0.009040,-0.007973,0.008440


---
### **Answering Question 1:**
####**Globals:**

In [13]:
#Multi-Factor regression function:
def multiFactorRegression(dataframes, names, y_col, avg_name):
  '''
  params:
  - y_col = (str) name of the column to use as y data
  - avg_name = (str) name to assign to the column with the calculated average excess return
  
  return:
  - Recap = (DataFrame) recap exhibit with the calculations done
  - Stats = (DataFrame) recap exhibit with the key statistics
  '''
  recap = {'Data': names, avg_name: [], 'Sharpe': [], 'R-squared': []}
  coefficients = ['Alpha', 'Beta_Mkt', 'Beta_SMB', 'Beta_HML', 'Beta_Term', 'Beta_Credit']
  stats = {}
  d = 0
  
  for df in dataframes:
    #reset:
    values = []
    ts = []
    ps = []
    #OLS regression model:
    #eq: (Ra-Rf) = alpha + beta1(Rm-Rf) + beta2(SMB) + beta3(HML) + beta4(Term) + beta5(Credit)
    #dependent:
    y = df[y_col].to_numpy()
    #independents:
    x = df[['Mkt_prem', 'SMB', 'HML', 'Term', 'Credit']].to_numpy()
    X = sm.add_constant(x)
    #fit the linear model:
    model = sm.OLS(y, X).fit()

    #excess return (ra-rf):
    avg_exc_ret = statistics.mean(df[y_col])
    #sharpe ratio = mean / std of excess rets:
    sharpe = statistics.mean(df[y_col]) / statistics.stdev(df[y_col])

    #populate recap:
    recap[avg_name].append(round(avg_exc_ret*100,4))
    recap['Sharpe'].append(round(sharpe, 4))
    recap['R-squared'].append(round(model.rsquared, 4))

    #populate stats for the current timeframe:
    for i in range(len(model.params)):
      values.append(round(model.params[i], 4))
      ts.append(round(model.tvalues[i], 4))
      ps.append(round(model.pvalues[i], 4))

    #convert stats for the current timeframe to a DataFrame:
    stats[names[d]] = pd.DataFrame(data={'Coefficient': coefficients, 'value': values, 't': ts, 'P_t': ps})
    d = d+1

  Recap = pd.DataFrame(recap)
  return Recap, stats

In [14]:
dataframes = [Data_30, Data_20, Data_10, Data_5]
names = ['30Y', '20Y', '10Y', '5Y']

####**Answers:**

**A) OLS Regression of a stock asset ('MSFT'):**

In [15]:
#get Recap data for Question 1.a:
Recap_1a, stats_1a = multiFactorRegression(dataframes, names, y_col='(Rs-Rf)', avg_name='Avg_(rs-rf)_%')
print("Question 1.a - Stock analysis (MSFT):\n")
Recap_1a

Question 1.a - Stock analysis (MSFT):



,Data,Avg_(rs-rf)_%,Sharpe,R-squared
0,30Y,1.6475,0.1903,0.3999
1,20Y,1.2699,0.1897,0.4318
2,10Y,2.2819,0.3892,0.4523
3,5Y,2.8304,0.5248,0.6656


In [16]:
stats_1a['30Y']

,Coefficient,value,t,P_t
0,Alpha,0.0249,2.4702,0.0140
1,Beta_Mkt,1.1378,13.2954,0.0000
2,Beta_SMB,-0.3527,-2.9610,0.0033
3,Beta_HML,-0.7236,-6.3442,0.0000
4,Beta_Term,-0.0203,-0.0625,0.9502
5,Beta_Credit,-1.5684,-1.6443,0.1010


In [17]:
stats_1a['20Y']

,Coefficient,value,t,P_t
0,Alpha,0.0122,1.2814,0.2013
1,Beta_Mkt,1.0597,13.0568,0.0000
2,Beta_SMB,-0.4708,-3.3133,0.0011
3,Beta_HML,-0.3183,-2.7253,0.0069
4,Beta_Term,-0.1303,-0.4171,0.6770
5,Beta_Credit,-0.4596,-0.5665,0.5716


In [18]:
stats_1a['10Y']

,Coefficient,value,t,P_t
0,Alpha,0.0335,1.7385,0.0848
1,Beta_Mkt,1.0150,9.4444,0.0000
2,Beta_SMB,-0.6318,-3.6991,0.0003
3,Beta_HML,-0.2979,-2.3137,0.0225
4,Beta_Term,-0.3547,-0.6596,0.5108
5,Beta_Credit,-2.0213,-1.1754,0.2423


In [19]:
stats_1a['5Y']

,Coefficient,value,t,P_t
0,Alpha,0.0513,1.7898,0.0791
1,Beta_Mkt,0.9586,9.7558,0.0000
2,Beta_SMB,-0.2923,-1.7796,0.0808
3,Beta_HML,-0.4374,-3.8561,0.0003
4,Beta_Term,-0.5438,-0.5664,0.5735
5,Beta_Credit,-3.8072,-1.4344,0.1572


**B) OLS Regression of a Portfolio:**

In [20]:
#get Recap data for Question 1.b:
Recap_1b, stats_1b = multiFactorRegression(dataframes, names, y_col='(Rp-Rf)', avg_name='Avg_(rp-rf)_%')
print("Question 2 - Portfolio analysis:\n")
Recap_1b

Question 2 - Portfolio analysis:



,Data,Avg_(rp-rf)_%,Sharpe,R-squared
0,30Y,0.9685,0.2054,0.8222
1,20Y,0.7917,0.1617,0.8617
2,10Y,1.0831,0.2685,0.8248
3,5Y,1.0896,0.2376,0.8605


In [21]:
stats_1b['30Y']

,Coefficient,value,t,P_t
0,Alpha,-0.0023,-0.7850,0.4330
1,Beta_Mkt,1.0090,39.7592,0.0000
2,Beta_SMB,-0.1085,-3.0711,0.0023
3,Beta_HML,0.2177,6.4364,0.0000
4,Beta_Term,0.0421,0.4364,0.6628
5,Beta_Credit,0.3698,1.3073,0.1920


In [22]:
stats_1b['20Y']

,Coefficient,value,t,P_t
0,Alpha,-0.0110,-3.2038,0.0015
1,Beta_Mkt,1.0301,35.1721,0.0000
2,Beta_SMB,-0.2026,-3.9502,0.0001
3,Beta_HML,0.2883,6.8398,0.0000
4,Beta_Term,0.1085,0.9630,0.3365
5,Beta_Credit,0.9016,3.0795,0.0023


In [23]:
stats_1b['10Y']

,Coefficient,value,t,P_t
0,Alpha,0.0096,1.2789,0.2035
1,Beta_Mkt,0.8825,21.1006,0.0000
2,Beta_SMB,-0.1680,-2.5278,0.0128
3,Beta_HML,0.3293,6.5728,0.0000
4,Beta_Term,0.0625,0.2987,0.7657
5,Beta_Credit,-1.1019,-1.6466,0.1024


In [24]:
stats_1b['5Y']

,Coefficient,value,t,P_t
0,Alpha,0.0058,0.3716,0.7116
1,Beta_Mkt,0.7983,14.7928,0.0000
2,Beta_SMB,-0.0500,-0.5539,0.5819
3,Beta_HML,0.4098,6.5780,0.0000
4,Beta_Term,0.2374,0.4502,0.6544
5,Beta_Credit,-0.6216,-0.4264,0.6715


---

## **Question 2**
#### **Get S&P 500 Futures data (e-Mini quotes -> ticker "ES=F"):**
NOTE: data are available only from November 2000, so the analysis will not be done for the 30Y period!

In [25]:
#download ES=F data (max = 20Y):
Spf_20 = pdr.get_data_yahoo('ES=F', start='2002-02-01', end='2022-01-31', interval='m')
Spf_20.reset_index(inplace=True)
print(Spf_20[['Date', 'Close', 'Adj Close']])

          Date    Close  Adj Close
0   2002-02-01  1107.00    1107.00
1   2002-03-01  1144.75    1144.75
2   2002-04-01  1076.00    1076.00
3   2002-05-01  1064.00    1064.00
4   2002-06-01   988.00     988.00
..         ...      ...        ...
200 2021-09-01  4297.75    4297.75
201 2021-10-01  4597.00    4597.00
202 2021-11-01  4566.25    4566.25
203 2021-12-01  4758.50    4758.50
204 2022-01-01  4504.25    4504.25

[205 rows x 3 columns]


In [26]:
#Merge data on Portfolio & Futures prices & returns (20Y only):
MeanPort = pd.DataFrame(data={'Date': MyPortfolio['Date'], 'Rp_adj': ret_mp.mean(axis = 1)})
Joined_20 = pd.merge(MeanPort, Spf_20[['Date', 'Adj Close']], on='Date')
Joined_20.rename(columns={'Adj Close': 'P_Spf'}, inplace=True)
Joined_20['RSpf'] = Joined_20['P_Spf'].pct_change(1)
Joined_20.fillna(0, inplace=True)
Joined_20

,Date,Rp_adj,P_Spf,RSpf
0,2002-02-01,0.004408,1107.00,0.000000
1,2002-03-01,0.036194,1144.75,0.034101
2,2002-04-01,-0.029138,1076.00,-0.060057
3,2002-05-01,-0.014578,1064.00,-0.011152
4,2002-06-01,-0.048670,988.00,-0.071429
...,...,...,...,...
200,2021-09-01,-0.013017,4297.75,-0.020902
201,2021-10-01,0.036900,4597.00,0.069629
202,2021-11-01,-0.021692,4566.25,-0.006689
203,2021-12-01,0.053288,4758.50,0.042102


In [27]:
#Split into smaller slices (10Y, 5Y):
Joined_10 = Joined_20.loc[(Joined_20['Date'] >= "2012-02-01")]
Joined_10.reset_index(drop=True, inplace=True)
Joined_5 = Joined_20.loc[(Joined_20['Date'] >= "2017-02-01")]
Joined_5.reset_index(drop=True, inplace=True)
#print:
Joined_10

,Date,Rp_adj,P_Spf,RSpf
0,2012-02-01,0.036516,1364.50,0.089421
1,2012-03-01,0.020922,1403.25,0.028399
2,2012-05-01,-0.082907,1309.25,-0.066987
3,2012-06-01,0.058235,1356.50,0.036089
4,2012-08-01,0.018741,1405.00,0.035754
...,...,...,...,...
97,2021-09-01,-0.013017,4297.75,-0.020902
98,2021-10-01,0.036900,4597.00,0.069629
99,2021-11-01,-0.021692,4566.25,-0.006689
100,2021-12-01,0.053288,4758.50,0.042102


In [28]:
Joined_5.head()

,Date,Rp_adj,P_Spf,RSpf
0,2017-02-01,0.043817,2362.75,0.056568
1,2017-03-01,0.015175,2359.25,-0.001481
2,2017-04-01,0.019800,2380.50,0.009007
3,2017-05-01,0.030997,2411.00,0.012812
4,2017-06-01,0.009040,2421.00,0.004148


---
### **Answering Question 2:**
#### **Globals:**

In [29]:
dataframes = [Joined_20, Joined_10, Joined_5]
names = ['20Y', '10Y', '5Y']

In [30]:
#One-Factor regression function:
def simple_regression(dataframes, names, x_col, y_col, avg_name):
  '''
  params:
  - x_col = (str) name of the column to use as x data
  - y_col = (str) name of the column to use as y data
  - avg_name = (str) name to assign to the column with the calculated average excess return
  
  return:
  - Recap = (DataFrame) recap exhibit with the calculations done
  '''
  recap = {'Data': names, 'Alpha': [], 'Beta': [], 'R-squared': [], 't(a)': [], 't(b)': [], 'P_t(a)': [], 'P_t(b)': []}

  for df in dataframes:
    #OLS regression model:
    #eq: y = alpha + beta*x
    x = df[x_col].to_numpy()
    y = df[y_col].to_numpy()
    X = sm.add_constant(x)
    #fit the linear model:
    model = sm.OLS(y, X).fit()

    #populate recap:
    recap['Alpha'].append(round(model.params[0], 4))
    recap['Beta'].append(round(model.params[1], 4))
    recap['R-squared'].append(round(model.rsquared, 4))
    recap['t(a)'].append(round(model.tvalues[0], 4))
    recap['P_t(a)'].append(round(model.pvalues[0], 4))
    recap['t(b)'].append(round(model.tvalues[1], 4))
    recap['P_t(b)'].append(round(model.pvalues[1], 4))

  Recap = pd.DataFrame(recap)
  return Recap

####**Answers:**
**A) Portfolio regression based on Futures price change:**

In [31]:
#get Recap data for Question 2:
Recap2 = simple_regression(dataframes, names, x_col='RSpf', y_col='Rp_adj', avg_name='Avg_Rp_%')
print("Question 2 - Portfolio regression based on Futures price change:\n")
Recap2

Question 2 - Portfolio regression based on Futures price change:



,Data,Alpha,Beta,R-squared,t(a),t(b),P_t(a),P_t(b)
0,20Y,0.0029,0.8354,0.6205,1.4712,18.2186,0.1428,0.0
1,10Y,0.0011,0.6884,0.5854,0.4662,11.8830,0.6421,0.0
2,5Y,0.0013,0.5844,0.5138,0.3461,7.2697,0.7307,0.0


**B) Hedging analysis:**

In [32]:
#Hedging:
invested_k = 1000000
data={'Data': names, 'Invested_K_USD': [], 'Last_S&P_Close_Price_USD': [], 'Optimal_num_futures': [], 'Rounded_num_futures': []}

for ind in Recap2.index:
  Period = dataframes[ind]
  opt_hedge_ratio = Recap2['Beta'].iloc[ind]
  latest_price_spf = Period['P_Spf'].iloc[-1]
  num_futures = round(opt_hedge_ratio * (invested_k / latest_price_spf), 2)
  rounded = int(math.modf(num_futures)[1])
  rounded = rounded+1 if rounded < num_futures else rounded
  #append:
  data['Invested_K_USD'].append(f"{invested_k:,.2f}")
  data['Last_S&P_Close_Price_USD'].append(f"{latest_price_spf:,.2f}")
  data['Optimal_num_futures'].append(num_futures)
  data['Rounded_num_futures'].append(rounded)

Hedging = pd.DataFrame(data)
Hedging

,Data,Invested_K_USD,Last_S&P_Close_Price_USD,Optimal_num_futures,Rounded_num_futures
0,20Y,"1,000,000.00","4,504.25",185.47,186
1,10Y,"1,000,000.00","4,504.25",152.83,153
2,5Y,"1,000,000.00","4,504.25",129.74,130
